In [1]:
import sys, os
if os.path.abspath(os.pardir) not in sys.path:
    sys.path.insert(0, os.path.abspath(os.pardir))
import CONFIG
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import pydicom
import random
from sklearn import preprocessing

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

In [4]:
ROOT = CONFIG.CFG.DATA.BASE
BATCH_SIZE = 64

In [5]:
train_df = pd.read_csv(os.path.join(ROOT, "train.csv"))
train_df.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker


In [7]:
train_df['min_week'] = train_df['Weeks']
train_df['min_week'] = train_df.groupby('Patient')['min_week'].transform('min')
train_df.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,min_week
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,-4
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker,-4
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker,-4
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker,-4
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker,-4


In [8]:
train_df['base_week'] = train_df['Weeks'] - train_df['min_week']
train_df.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,min_week,base_week
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,-4,0
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker,-4,9
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker,-4,11
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker,-4,13
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker,-4,15


In [10]:
min_max_scaler = preprocessing.MinMaxScaler()

In [11]:
lstm_input = train_df[['base_week', 'FVC', 'Age']].values
lstm_input_scaled = torch.tensor(min_max_scaler.fit_transform(lstm_input)).float()
train_df[['base_week', 'FVC', 'Age']] = lstm_input_scaled

In [12]:
train_df.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,min_week,base_week
0,ID00007637202177411956430,-4,0.267050,58.253649,0.769231,Male,Ex-smoker,-4,0.000000
1,ID00007637202177411956430,5,0.248923,55.712129,0.769231,Male,Ex-smoker,-4,0.142857
2,ID00007637202177411956430,7,0.221464,51.862104,0.769231,Male,Ex-smoker,-4,0.174603
3,ID00007637202177411956430,9,0.236360,53.950679,0.769231,Male,Ex-smoker,-4,0.206349
4,ID00007637202177411956430,11,0.222900,52.063412,0.769231,Male,Ex-smoker,-4,0.238095


In [13]:
class OSICLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(OSICLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.lc = nn.Linear(hidden_size, 50)
        self.lc2 = nn.Linear(50, output_size)

    def forward(self, X):
        out, _ = self.lstm(X)
        out = self.lc(out)
        out = self.lc2(out)
        return out

In [19]:
model = OSICLSTM(3, 200, 3, 3)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [20]:
ALL_PATIENTS = train_df["Patient"].unique().tolist()

In [21]:
model.train()

OSICLSTM(
  (lstm): LSTM(3, 200, num_layers=3, batch_first=True)
  (lc): Linear(in_features=200, out_features=50, bias=True)
  (lc2): Linear(in_features=50, out_features=3, bias=True)
)

In [22]:
for patient in ALL_PATIENTS:
    patient_data = train_df[train_df["Patient"] == patient][['min_week', 'FVC', 'Age']].values
    patient_data = torch.tensor(patient_data).float()

    for data in patient_data:
        out = model(data.view(1,1,-1))
        print(out)
        break
    break

tensor([[[-0.1140, -0.1158,  0.0018]]], grad_fn=<AddBackward0>)


In [23]:
for epoch in range(1000):
    total_loss = 0
    for patient in ALL_PATIENTS:
        patient_loss = 0
        model.zero_grad()

        patient_data = train_df[train_df["Patient"] == patient][['min_week', 'FVC', 'Age']].values
        patient_data = torch.tensor(patient_data).float()

        loss = 0
        for i in range(patient_data.size()[0] - 1):
            out = model(patient_data[i].view(1,1,-1))
            loss += loss_function(out.view(3), patient_data[i+1])
            patient_loss += loss.item()
        # for data in patient_data:
        #     out = model(data.view(1, 1, -1))
        #     print(out)
        loss.backward()
        optimizer.step()
        total_loss += patient_loss/patient_data.shape[0]
    print(f"Epoch {epoch}, loss={total_loss}")

Epoch 0, loss=23900.01107638649
Epoch 1, loss=15055.669008816121
Epoch 2, loss=12559.621976547141
Epoch 3, loss=6961.615223087998
Epoch 4, loss=4881.21737738177
Epoch 5, loss=24271.688548769223
Epoch 6, loss=46815.72071284326
Epoch 7, loss=31920.383186828927
Epoch 8, loss=26907.299734701424
Epoch 9, loss=15846.584536164532
Epoch 10, loss=23505.28411441258
Epoch 11, loss=26480.95775312662
Epoch 12, loss=12427.572465533747
Epoch 13, loss=8288.950672816403
Epoch 14, loss=23034.282620031518
Epoch 15, loss=15574.389137453876
Epoch 16, loss=20065.05176988326
Epoch 17, loss=12505.061204730757
Epoch 18, loss=8990.748810592811
Epoch 19, loss=10279.831789966835
Epoch 20, loss=4033.331575033603
Epoch 21, loss=6867.858081304442
Epoch 22, loss=8520.852169791819
Epoch 23, loss=7478.322513859993
Epoch 24, loss=9490.116198775893
Epoch 25, loss=7426.826837332811
Epoch 26, loss=17079.889476676344
Epoch 27, loss=26825.688856519184
Epoch 28, loss=16988.35590349339
Epoch 29, loss=25644.89848824172
Epoch 30

In [96]:
sample_u = torch.tensor([[-12, 3020, 73]])
sample = torch.tensor(min_max_scaler.transform(sample_u)).float()
sample_u[:, 0]

tensor([-12])

In [97]:
with torch.no_grad():
    for i in range(120):
        out = model(sample.view(1,1,-1))
        out = min_max_scaler.inverse_transform(out.squeeze(dim=0))
        print(out)
        prev_zero = sample_u[:,0]
        prev_three = sample_u[:,2]
        out[:, 0] = prev_zero + i
        out[:, 2] = prev_three
        print(out, "\n")
        sample = torch.tensor(min_max_scaler.transform(out)).float()

[[ -14.31420559 2945.78741264   73.12457055]]
[[ -12.         2945.78741264   73.        ]] 

[[ -14.38764644 2874.82694507   73.13517064]]
[[ -11.         2874.82694507   73.        ]] 

[[ -13.16220379 2808.05978775   73.1353845 ]]
[[ -10.         2808.05978775   73.        ]] 

[[ -11.92862207 2744.85890365   73.13483357]]
[[  -9.         2744.85890365   73.        ]] 

[[ -10.68775672 2684.68925214   73.13386887]]
[[  -8.         2684.68925214   73.        ]] 

[[  -9.44036448 2627.08802807   73.13275075]]
[[  -7.         2627.08802807   73.        ]] 

[[  -8.1870088  2571.65270555   73.13167447]]
[[  -6.         2571.65270555   73.        ]] 

[[  -6.92818731 2518.03074229   73.13079345]]
[[  -5.         2518.03074229   73.        ]] 

[[  -5.66428661 2465.91077852   73.13021231]]
[[  -4.         2465.91077852   73.        ]] 

[[  -4.39559871 2415.01715708   73.13001239]]
[[  -3.         2415.01715708   73.        ]] 

[[  -3.12236214 2365.10494161   73.13024718]]
[[-2.00000000e

In [ ]:
for i in range(5):
    print(sample)
    out = model(sample.view(1, 1, -1))
    print(out)
    sample = out

In [ ]:
with torch.no_grad():
    input = torch.randn(2, 5, 4)
    print(input)
    print()
    out = model(input)
    print(out)
    print(out.shape)